# SVM with Class Imbalance Handling
Support Vector Machines (SVMs) search for the decision boundary that maximises the margin between classes. For imbalanced datasets the classifier can ignore the minority class, so we balance the training data by **oversampling** minority examples. After fitting a linear-kernel SVM we evaluate precision, recall, F1-score, and the confusion matrix to confirm improvements for the minority class.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils import resample

In [ ]:
# Generate an imbalanced dataset (spam detection style)
X, y = make_classification(
    n_samples=300,
    n_features=10,
    n_informative=4,
    n_redundant=2,
    n_clusters_per_class=1,
    weights=[0.85, 0.15],
    class_sep=1.2,
    random_state=42
)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)

print("Class balance before oversampling:")
print(pd.Series(y_train).value_counts())

In [ ]:
# Oversample minority class to balance the dataset
train_df = pd.DataFrame(X_train)
train_df["label"] = y_train
majority = train_df[train_df["label"] == 0]
minority = train_df[train_df["label"] == 1]

minority_upsampled = resample(
    minority,
    replace=True,
    n_samples=len(majority),
    random_state=42
)

balanced_df = pd.concat([majority, minority_upsampled])
X_balanced = balanced_df.drop(columns="label").to_numpy()
y_balanced = balanced_df["label"].to_numpy()

print("Class balance after oversampling:")
print(pd.Series(y_balanced).value_counts())

In [ ]:
svm_model = SVC(kernel="linear", random_state=42)
svm_model.fit(X_balanced, y_balanced)

y_pred = svm_model.predict(X_test)

print("Confusion matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nDetailed report:")
print(classification_report(y_test, y_pred, target_names=["normal", "spam"]))